### Class Decorators

In [1]:
def savings(cls):
    cls.account_type = 'savings'
    return cls

def checking(cls):
    cls.account_type = 'checking'
    return cls

In [2]:
class Account:
    pass

@savings
class Bank1Savings(Account):
    pass

@savings
class Bank2Savings(Account):
    pass

@checking
class Bank1Checking(Account):
    pass

@checking
class Bank2Checking(Account):
    pass

In [3]:
Bank1Savings.__dict__

mappingproxy({'__module__': '__main__',
              '__doc__': None,
              'account_type': 'savings'})

In [4]:
Bank2Checking.__dict__

mappingproxy({'__module__': '__main__',
              '__doc__': None,
              'account_type': 'checking'})

In [5]:
def account_type(type_):
    def decorator(cls):
        cls.account_type = type_
        return cls
    return decorator

In [6]:
@account_type('Savings')
class Bank1Savings:
    pass

In [7]:
@account_type('Checking')
class Bank1Checking:
    pass

In [8]:
Bank1Savings.__dict__

mappingproxy({'__module__': '__main__',
              '__dict__': <attribute '__dict__' of 'Bank1Savings' objects>,
              '__weakref__': <attribute '__weakref__' of 'Bank1Savings' objects>,
              '__doc__': None,
              'account_type': 'Savings'})

In [9]:
Bank1Checking.__dict__

mappingproxy({'__module__': '__main__',
              '__dict__': <attribute '__dict__' of 'Bank1Checking' objects>,
              '__weakref__': <attribute '__weakref__' of 'Bank1Checking' objects>,
              '__doc__': None,
              'account_type': 'Checking'})

In [10]:
def hello(cls):
    cls.hello = lambda self: f'{self} says hello!'
    return cls

In [11]:
@hello
class Person:
    def __init__(self, name):
        self.name = name

    def __str__(self):
        return self.name

In [12]:
vars(Person)

mappingproxy({'__module__': '__main__',
              '__init__': <function __main__.Person.__init__(self, name)>,
              '__str__': <function __main__.Person.__str__(self)>,
              '__dict__': <attribute '__dict__' of 'Person' objects>,
              '__weakref__': <attribute '__weakref__' of 'Person' objects>,
              '__doc__': None,
              'hello': <function __main__.hello.<locals>.<lambda>(self)>})

In [13]:
p = Person("Guido")

In [14]:
p.hello()

'Guido says hello!'

In [15]:
from functools import wraps

In [16]:
def func_logger(fn):
    @wraps(fn)
    def inner(*args, **kwargs):
        result = fn(*args, **kwargs)
        print(f'Log: {fn.__qualname__}({args}, {kwargs}) = {result}')
        return result
    return inner

In [17]:
class Person:
    @func_logger
    def __init__(self, name, age):
        self.name = name
        self.age = age

    @func_logger
    def greet(self):
        return f'Hello, my name is {self.name}, and I am {self.age} years ols'

In [18]:
p = Person('John', 78)

Log: Person.__init__((<__main__.Person object at 0x7f312412c9b0>, 'John', 78), {}) = None


In [19]:
p.greet()

Log: Person.greet((<__main__.Person object at 0x7f312412c9b0>,), {}) = Hello, my name is John, and I am 78 years ols


'Hello, my name is John, and I am 78 years ols'

In [20]:
def class_logger(cls):
    for name, obj in vars(cls).items():
        if callable(obj):
            print('decorating:', cls, name)
            setattr(cls, name, func_logger(obj))
    return cls

In [21]:
@class_logger
class Person:
    def __init__(self, name, age):
        self.name = name
        self.age = age

    def greet(self):
        return f'Hello, my name is {self.name}, and I am {self.age} years ols'

decorating: <class '__main__.Person'> __init__
decorating: <class '__main__.Person'> greet


In [22]:
p = Person('John' ,78)

Log: Person.__init__((<__main__.Person object at 0x7f312412d340>, 'John', 78), {}) = None


In [23]:
p.greet()

Log: Person.greet((<__main__.Person object at 0x7f312412d340>,), {}) = Hello, my name is John, and I am 78 years ols


'Hello, my name is John, and I am 78 years ols'

In [24]:
@class_logger
class Person:
    @staticmethod
    def static_method():
        print('static method invoked...')

    @classmethod
    def cls_method(cls):
        print(f'cls_method invoked for {cls}...')

    def instance_method(self):
        print(f'instance_method invoked for {self}...')

decorating: <class '__main__.Person'> static_method
decorating: <class '__main__.Person'> instance_method


In [25]:
Person.__dict__

mappingproxy({'__module__': '__main__',
              'static_method': <function __main__.Person.static_method()>,
              'cls_method': <classmethod(<function Person.cls_method at 0x7f31240f6340>)>,
              'instance_method': <function __main__.Person.instance_method(self)>,
              '__dict__': <attribute '__dict__' of 'Person' objects>,
              '__weakref__': <attribute '__weakref__' of 'Person' objects>,
              '__doc__': None})

In [26]:
Person.__dict__['static_method']

<function __main__.Person.static_method()>

In [27]:
callable(Person.__dict__['static_method'])

True

In [28]:
Person.__dict__['cls_method']

<classmethod(<function Person.cls_method at 0x7f31240f6340>)>

In [29]:
callable(Person.__dict__['cls_method'])

False

In [30]:
class Person:
    @func_logger
    @classmethod
    def class_method(cls):
        pass

In [31]:
Person.class_method()

TypeError: 'classmethod' object is not callable

In [32]:
class Person:
    @staticmethod
    def static_method():
        pass

    @classmethod
    def class_method():
        pass

In [33]:
type(Person.__dict__['static_method'])

staticmethod

In [34]:
type(Person.__dict__['class_method'])

classmethod

In [35]:
Person.__dict__['static_method'].__func__

<function __main__.Person.static_method()>

In [ ]:
def class_logger(cls):
    for name, obj in vars(cls).items():
        if callable(obj):
            print('decorating callable', cls, name)
            original_func = obj
            decorated_func = func_logger(original_func)
            setattr(cls, name, decorated_func)
        elif isinstance(obj, staticmethod):
            print('decorating static method', cls, name)
            original_func = obj.__func__
            decorated_func - func_logger(original_func)
            method = staticmethod(decorated_func)
            setattr(cls, name, method)
        elif isinstance(obj, classmethod):
            print('decorating class method', cls, name)
            original_func = obj.__func__
            decorated_func - func_logger(original_func)
            method = classmethod(decorated_func)
            setattr(cls, name, method)
    return cls
